# Sensitivity Analysis Quickstart

This notebook illustrates how to drive the `run_sensitivity_analysis` tool via
the MCP bridge. It walks through loading a model, preparing parameter deltas,
launching the analysis, and inspecting the resulting PK deltas.

## 1. Configure API access

Ensure the MCP bridge is running and set `MCP_BASE_URL` / `MCP_TOKEN` in your
environment. Sensitivity analysis is a critical tool so a confirmation header
is required.

In [ ]:
import json
import os
import time
from pathlib import Path

import requests

BASE_URL = os.environ.get("MCP_BASE_URL", "http://localhost:8000")
TOKEN = os.environ["MCP_TOKEN"]
HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json",
    "confirm": true,
}

def _post(path: str, payload: dict) -> requests.Response:
    url = f"{BASE_URL}{path}"
    response = requests.post(url, headers=HEADERS, data=json.dumps(payload), timeout=60)
    response.raise_for_status()
    return response


## 2. Load the baseline simulation

Re-use the single-subject demo model shipped with the repository.

In [ ]:
model_path = "tests/fixtures/demo.pkml"
simulation_id = "sensitivity-demo"

load_response = _post("/load_simulation", {"filePath": model_path, "simulationId": simulation_id})
load_response.json()


## 3. Prepare parameter deltas

Each parameter entry lists the percentage changes to apply relative to the
baseline. Optionally include absolute bounds and baseline overrides when the
adapter cannot infer them.

In [ ]:
parameters = [
    {
        "path": "Organism|Weight",
        "deltas": [-0.25, 0.0, 0.25],
        "unit": "kg",
    },
    {
        "path": "Organism|Height",
        "deltas": [-0.1, 0.1],
        "unit": "cm",
    },
]

sensitivity_payload = {
    "modelPath": model_path,
    "simulationId": simulation_id,
    "parameters": parameters,
    "includeBaseline": True,
    "pollIntervalSeconds": 0.5,
    "metrics": ["Cmax", "Tmax", "AUC"],
}


## 4. Launch the sensitivity analysis

The tool responds immediately with a job identifier. Progress updates stream
through `get_job_status` and each scenario result is persisted in the job
record.

In [ ]:
sensitivity_response = _post("/run_sensitivity_analysis", sensitivity_payload)
job_id = sensitivity_response.json()["jobId"]
job_id


## 5. Poll for completion and inspect the report

When the analysis finishes, the response includes a structured report with
scenario-level metrics and a CSV attachment (base64-encoded) mirroring the
filesystem artefact.

In [ ]:
status_payload = {"jobId": job_id}
while True:
    status = _post("/get_job_status", status_payload).json()["job"]
    print(status["status"], status.get("attempts"))
    if status["status"].lower() not in {"queued", "running"}:
        break
    time.sleep(1.0)

report_response = _post("/get_job_status", status_payload)
report = report_response.json()["job"].get("resultHandle", {})
report


## 6. Retrieve PK deltas directly

Call the tool endpoint to obtain the structured report and CSV content in one
response. Persist the CSV locally for downstream plotting.

In [ ]:
import base64

tool_payload = {"jobId": job_id}
analysis_response = _post("/mcp/call_tool", {"tool": "run_sensitivity_analysis", "arguments": tool_payload, "critical": True})
analysis_result = analysis_response.json()["structuredContent"]

csv_attachment = analysis_result["csv"]
csv_path = Path("sensitivity_results.csv")
csv_path.write_bytes(base64.b64decode(csv_attachment["data"]))
csv_path
